In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime as dt

# --------------------------------------------------
# 1. CONEXIÓN Y EXTRACCIÓN
# --------------------------------------------------
try:
    oltp_conn = mysql.connector.connect(
        host="192.168.0.103",
        port=3307,
        user="etl_user",
        password="TuPasswordFuerte",
        database="db_gestion"
    )
    oltp_cursor = oltp_conn.cursor(dictionary=True)
    
    print("--- FASE 1: EXTRACCIÓN ---")
    oltp_cursor.callproc("obtener_todo")
    result_sets = []
    
    for res in oltp_cursor.stored_results():
        result_sets.append(pd.DataFrame(res.fetchall()))

    # Desempaquetar DataFrames
    cliente_df, equipo_df, empleado_df, estadistica_df, proyecto_df, tarea_df, asignacion_df, incidente_df = result_sets

    oltp_cursor.close()
    oltp_conn.close()
    print(f"Datos extraídos exitosamente. {len(proyecto_df)} proyectos totales encontrados.")

except Exception as e:
    print(f"Error CRÍTICO en extracción OLTP: {e}")
    exit()

# 2. TRANSFORMACIÓN Y LÓGICA
def safe_strip_cols(df):
    if not df.empty:
        df.columns = [str(col).strip() for col in df.columns]
    return df

def safe_index(df, colname):
    if colname in df.columns and not df.empty:
        df = df.dropna(subset=[colname])
        df = df.loc[~df[colname].duplicated()]
        return df.set_index(colname, drop=False)
    return pd.DataFrame()

def strict_lookup(df_indexed, key):
    try:
        if key in df_indexed.index:
            return df_indexed.loc[key]
        return None
    except:
        return None

def safe_int(val):
    try: return int(val)
    except: return 0

def safe_float(val):
    try: return float(val)
    except: return 0.0

def descomponer_fecha(fecha, id_tiempo):
    if fecha is None or pd.isna(fecha):
        fecha = dt.now()
    if isinstance(fecha, str):
        try: fecha = dt.strptime(fecha, "%Y-%m-%d")
        except: fecha = dt.now()
    elif isinstance(fecha, pd.Timestamp):
        fecha = fecha.to_pydatetime()
    
    return {
        "idTiempo": id_tiempo,
        "fecha_completa": fecha.strftime("%Y-%m-%d"),
        "anio": fecha.year,
        "trimestre": (fecha.month - 1) // 3 + 1,
        "mes": fecha.month,
        "semana": fecha.isocalendar()[1],
        "dia": fecha.day
    }

# Limpieza básica de columnas
for df in [cliente_df, equipo_df, empleado_df, proyecto_df, tarea_df, asignacion_df, incidente_df]:
    safe_strip_cols(df)

# Filtro de proyectos finalizados y cancelados
if not proyecto_df.empty and 'Estado' in proyecto_df.columns:
    count_original = len(proyecto_df)
    # Filtramos el DataFrame para dejar solo FINALIZADOS y CANCELADOS
    proyecto_df = proyecto_df[proyecto_df['Estado'].isin(['FINALIZADO', 'CANCELADO'])]
    count_final = len(proyecto_df)
    print(f"Filtro aplicado: De {count_original} proyectos, quedan {count_final} FINALIZADOS o CANCELADOS.")

    # Solo los que pertenecen a proyectos FINALIZADOS o CANCELADOS
    if not incidente_df.empty and 'Proyecto_idProyecto' in incidente_df.columns:
        count_incidentes_original = len(incidente_df)
        ids_proyectos_validos = set(proyecto_df['idProyecto'])
        incidente_df = incidente_df[incidente_df['Proyecto_idProyecto'].isin(ids_proyectos_validos)]
        count_incidentes_final = len(incidente_df)
        print(f"Incidentes filtrados: De {count_incidentes_original} incidentes, quedan {count_incidentes_final} de proyectos FINALIZADOS/CANCELADOS.")


cliente_by_id = safe_index(cliente_df, "idCliente")
equipo_by_id = safe_index(equipo_df, "idEquipo")
empleado_by_id = safe_index(empleado_df, "idEmpleado")
proyecto_by_id = safe_index(proyecto_df, "idProyecto")
tarea_by_id = safe_index(tarea_df, "idTarea")

if not asignacion_df.empty and not tarea_df.empty:
    asig_completa_df = asignacion_df.merge(
        tarea_df, left_on='Tarea_idTarea', right_on='idTarea', how='left', suffixes=('_asig', '_tarea')
    )
else:
    asig_completa_df = pd.DataFrame()

def obtener_metricas_proyecto(pid):
    m = {"tareas_auto": 0, "tareas_reutil": 0, "horas_est": 0.0, "horas_real": 0.0, "avance": 0.0, "costo_defecto": 0.0}
    
    if not asig_completa_df.empty:
        filtro_proy = asig_completa_df[asig_completa_df['Proyecto_idProyecto'] == pid]
        if not filtro_proy.empty:
            m["tareas_auto"] = safe_int(filtro_proy[filtro_proy['EsAutomatizacion'] == 1].shape[0])
            m["tareas_reutil"] = safe_int(filtro_proy[filtro_proy['EsReutilizado'] == 1].shape[0])
            
            col_est = 'Horas_estimadas_asig' if 'Horas_estimadas_asig' in filtro_proy.columns else 'Horas_estimadas'
            m["horas_est"] = safe_float(filtro_proy[col_est].sum())
            m["horas_real"] = safe_float(filtro_proy['Horas_reales'].sum())
            
            total_tareas = filtro_proy.shape[0]
            col_estado = 'Estado_tarea' if 'Estado_tarea' in filtro_proy.columns else 'Estado'
            tareas_compl = filtro_proy[filtro_proy[col_estado] == 'COMPLETADA'].shape[0]
            if total_tareas > 0:
                m["avance"] = (tareas_compl / total_tareas) * 100.0

    if not incidente_df.empty:
        incidentes_proy = incidente_df[incidente_df['Proyecto_idProyecto'] == pid]
        m["costo_defecto"] = safe_float(incidentes_proy['CostoCorreccion'].sum())

    return m

cache_metricas = {}
if not proyecto_df.empty:
    for pid in proyecto_df['idProyecto'].unique():
        cache_metricas[pid] = obtener_metricas_proyecto(pid)

def get_empleado_y_equipo(pid, tid):
    if asignacion_df.empty: return None, None
    
    # Relación de tarea y proyecto
    if tid and tid != 0:
        match = asignacion_df[(asignacion_df["Proyecto_idProyecto"] == pid) & (asignacion_df["Tarea_idTarea"] == tid)]
        if not match.empty:
            emp = strict_lookup(empleado_by_id, match.iloc[0]["Empleado_idEmpleado"])
            if emp is not None:
                eq = strict_lookup(equipo_by_id, emp["Equipo_idEquipo"])
                if eq is not None: return emp, eq

    # Relacionar proyecto y equipo
    match_proy = asignacion_df[asignacion_df["Proyecto_idProyecto"] == pid]
    if not match_proy.empty:
        emp = strict_lookup(empleado_by_id, match_proy.iloc[0]["Empleado_idEmpleado"])
        if emp is not None:
            eq = strict_lookup(equipo_by_id, emp["Equipo_idEquipo"])
            if eq is not None: return None, eq
            
    return None, None

# 3. CARGA
dw_conn = None
try:
    dw_conn = mysql.connector.connect(
        host="192.168.0.103", port=3307, user="etl_user", password="TuPasswordFuerte", database="db_soporte"
    )

    dw_conn.autocommit = False
    dw_cursor = dw_conn.cursor()
    
    print("\n--- FASE 2: CARGA AL DATA WAREHOUSE ---")
    
    registros_insertados = 0
    registros_omitidos = 0

    if incidente_df.empty:
        print("No hay incidentes (de proyectos finalizados/cancelados) para procesar.")
    else:
        # Inicio del bloque transaccional
        for index, inc in incidente_df.iterrows():
            try:
                pid = inc.get("Proyecto_idProyecto")
                tid = inc.get("idTarea")
                id_incidente_oltp = inc.get('idIncidente')

                # VALIDACIONES
                proy = strict_lookup(proyecto_by_id, pid)
                if proy is None:
                    print(f"[OMITIDO] Incidente {id_incidente_oltp}: Proyecto {pid} no esta Finalizafo o Cancelado.")
                    registros_omitidos += 1
                    continue

                cli = strict_lookup(cliente_by_id, proy.get("Cliente_idCliente"))
                if cli is None:
                    print(f"[OMITIDO] Incidente {id_incidente_oltp}: Cliente no encontrado.")
                    registros_omitidos += 1
                    continue

                emp_encontrado, eq_encontrado = get_empleado_y_equipo(pid, tid)
                if eq_encontrado is None:
                    print(f"[OMITIDO] Incidente {id_incidente_oltp}: No se encontró Equipo via asignación")
                    registros_omitidos += 1
                    continue
                
                # DATOS
                tarea = strict_lookup(tarea_by_id, tid)
                mets = cache_metricas.get(pid, {"tareas_auto":0, "tareas_reutil":0, "horas_est":0.0, "horas_real":0.0, "avance":0.0, "costo_defecto":0.0})
                
                # PRIORIDAD
                prioridad_num = None
                if tarea is not None:
                    p_text = str(tarea.get("Prioridad", "")).strip().upper()
                    prioridad_num = {"BAJA": 1, "MEDIA": 2, "ALTA": 3, "CRITICA": 4, "CRÍTICA": 4}.get(p_text, 0)

                # Preparación de parámetros
                # Determinar el estado real del proyecto
                estado_proy = str(proy.get("Estado", "")).strip().upper()
                if estado_proy == "FINALIZADO":
                    idEstado = 1
                    nombreEstado = "FINALIZADO"
                elif estado_proy == "CANCELADO":
                    idEstado = 3
                    nombreEstado = "CANCELADO"
                else:
                    idEstado = 1
                    nombreEstado = estado_proy 

                tinfo = descomponer_fecha(proy.get("Fecha_fin_real"), pid)
                cinfo = {"idCalidad": 1, "severidad_defecto": str(inc.get("Severidad", "")), "tipo_incidente": "INCIDENTE", "cert_calidad": 0}
                
                params = [
                    # Cliente
                    int(cli["idCliente"]), str(cli["Nombre"]), str(cli["Email"]), str(cli["Telefono"]), str(cli["Industria"]), safe_float(cli.get("MetricaClienteInicial")),
                    # Equipo
                    int(eq_encontrado["idEquipo"]), str(eq_encontrado["Nombre"]), safe_int(eq_encontrado["Activo"]),
                    # Empleado
                    int(emp_encontrado["idEmpleado"]) if emp_encontrado is not None else 0, 
                    str(emp_encontrado["Nombre"]) if emp_encontrado is not None else "", 
                    str(emp_encontrado["Email"]) if emp_encontrado is not None else "", 
                    safe_float(emp_encontrado["Salario"]) if emp_encontrado is not None else 0.0, 
                    safe_float(emp_encontrado["SalarioxHora"]) if emp_encontrado is not None else 0.0, 
                    int(emp_encontrado["Equipo_idEquipo"]) if emp_encontrado is not None else 0,
                    # Estado
                    int(idEstado), str(nombreEstado),
                    # Proyecto
                    int(proy["idProyecto"]), str(proy["Nombre"]), str(proy["Tipo"]), str(proy["Descripcion"]), 
                    safe_float(proy.get("Presupuesto")), safe_float(proy.get("Costo_real")), 
                    safe_float(proy.get("MetricaClienteFinal")), safe_int(proy["CertificacionSeguridad"]), 
                    proy.get("Fecha_inicio"), proy.get("Fecha_fin_estimada"), proy.get("Fecha_fin_real"), 
                    int(proy["Cliente_idCliente"]), int(eq_encontrado["idEquipo"]), int(idEstado),
                    # Tarea
                    int(tarea["idTarea"]) if tarea is not None else 0, 
                    str(tarea["Titulo"]) if tarea is not None else "", 
                    str(tarea["Descripcion"]) if tarea is not None else "", 
                    tarea.get("Fecha_creacion") if tarea is not None else None, 
                    tarea.get("Fecha_fin_estimada") if tarea is not None else None, 
                    tarea.get("Fecha_fin_real") if tarea is not None else None, 
                    prioridad_num,
                    safe_int(tarea.get("EsAutomatizacion")) if tarea is not None else 0, 
                    safe_int(tarea.get("EsReutilizado")) if tarea is not None else 0, 
                    int(pid),
                    # Tiempo
                    int(tinfo["idTiempo"]), tinfo["fecha_completa"], int(tinfo["anio"]), int(tinfo["trimestre"]), int(tinfo["mes"]), int(tinfo["semana"]), int(tinfo["dia"]),
                    # Calidad
                    int(cinfo["idCalidad"]), str(cinfo["severidad_defecto"]), str(cinfo["tipo_incidente"]), int(cinfo["cert_calidad"]),
                    # FK Hechos
                    int(pid), int(pid), int(cli["idCliente"]), int(eq_encontrado["idEquipo"]), int(tinfo["idTiempo"]), int(idEstado),
                    # Métricas Hechos
                    safe_float(proy.get("Presupuesto")), safe_float(proy.get("Costo_real")), (safe_float(proy.get("Presupuesto")) - safe_float(proy.get("Costo_real"))), 
                    safe_float(cli.get("MetricaClienteInicial")), safe_float(proy.get("MetricaClienteFinal")),
                    mets["tareas_auto"], mets["tareas_reutil"], len(incidente_df[incidente_df["Proyecto_idProyecto"] == pid]),
                    mets["costo_defecto"], mets["avance"], mets["horas_est"], mets["horas_real"],
                    # Info Incidente
                    int(inc["idIncidente"]), int(pid), int(tarea["idTarea"]) if tarea is not None else 0, int(cinfo["idCalidad"]), 
                    inc.get("Fecha_reporte"), str(inc.get("Severidad")), str(inc.get("Estado")), safe_float(inc.get("CostoCorreccion"))
                ]

                # Inserción de datos llamando al procedimiento almacenado
                dw_cursor.callproc("cargar_todo_dw", params)
                registros_insertados += 1
            
            except Exception as e_row:
                raise Exception(f"Error procesando Incidente {inc.get('idIncidente')}: {e_row}")

        dw_conn.commit()
        print(f"\nSe han insertado {registros_insertados} registros correctamente.")

except Exception as e_gral:
    # Si ocurre cualquier error en el proceso ocurre ROLLBACK
    print(f"\nError detectado: {e_gral}")
    if dw_conn and dw_conn.is_connected():
        dw_conn.rollback()
        print("ROLLBACK EJECUTADO: Se han deshecho todos los cambios. La BD está limpia.")
    else:
        print("No se pudo ejecutar Rollback (conexión perdida o no iniciada).")

finally:
    # Para cerrar conexión
    if dw_conn and dw_conn.is_connected():
        dw_cursor.close()
        dw_conn.close()
        print("Conexión cerrada.")


--- FASE 1: EXTRACCIÓN ---
Datos extraídos exitosamente. 275 proyectos totales encontrados.
Filtro aplicado: De 275 proyectos, quedan 82 FINALIZADOS o CANCELADOS.
Incidentes filtrados: De 600 incidentes, quedan 130 de proyectos FINALIZADOS/CANCELADOS.

--- FASE 2: CARGA AL DATA WAREHOUSE ---


C:\Users\Torpid18\AppData\Local\Temp\ipykernel_24624\3059521724.py:23: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for res in oltp_cursor.stored_results():


[OMITIDO] Incidente 363: No se encontró Equipo via asignación
[OMITIDO] Incidente 371: No se encontró Equipo via asignación
[OMITIDO] Incidente 380: No se encontró Equipo via asignación
[OMITIDO] Incidente 381: No se encontró Equipo via asignación
[OMITIDO] Incidente 384: No se encontró Equipo via asignación
[OMITIDO] Incidente 399: No se encontró Equipo via asignación
[OMITIDO] Incidente 415: No se encontró Equipo via asignación
[OMITIDO] Incidente 420: No se encontró Equipo via asignación
[OMITIDO] Incidente 425: No se encontró Equipo via asignación
[OMITIDO] Incidente 427: No se encontró Equipo via asignación
[OMITIDO] Incidente 429: No se encontró Equipo via asignación
[OMITIDO] Incidente 438: No se encontró Equipo via asignación
[OMITIDO] Incidente 442: No se encontró Equipo via asignación
[OMITIDO] Incidente 447: No se encontró Equipo via asignación
[OMITIDO] Incidente 450: No se encontró Equipo via asignación
[OMITIDO] Incidente 455: No se encontró Equipo via asignación
[OMITIDO